In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show

In [2]:
df = pd.read_excel('HexaPropsAssay.xlsx')

# print(df[~df.applymap(lambda x: not isinstance(x, (int, float)))])

In [22]:
# def norm(df_column_name):
#     input_array=np.array(df[df_column_name])
#     if np.isnan(input_array).any():
#         pass #do thing if the input_array contains any non-integer items
#     else:
#         return (input_array - np.min(input_array))/(np.max(input_array) - np.min(input_array))

# from sklearn import preprocessing
# x = df.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# ndf = pandas.DataFrame(x_scaled)

# ndf = pd.DataFrame({x:norm(x) for x in df.columns if x != 'Name'})
factors = []
for x in df.columns:
    factors.append(x)

hm = figure(title="Categorical Heatmap", tools="hover", toolbar_location=None,
            x_range=factors, y_range=factors)
colors = [
    "#0B486B", "#79BD9A", "#CFF09E",
    "#79BD9A", "#0B486B", "#79BD9A",
    "#CFF09E", "#79BD9A", "#0B486B"
]
a = df.values
p = figure(x_range=factors, y_range=(0, 2))
p.image(image=[a], x=0, y=0, dw=2, dh=2, palette="Spectral11")
show(p)
# output_notebook()